# 1. Business Understanding

The aim of this notebook is to process the LinkedIn profiles of individual employees into company-level features. The features are designed on the basis of the literature research of the corresponding master thesis. This notebook forms the basis for further analyses, which are described in the notebooks "Default_final_2308" and "Downgrade_final_2308".

# 2. Load data and prepare libaries

With the use of Chat GPD, comments have been added for readability.

## 2.1 Import libaries

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

## 2.2 Load datasets

In [ ]:
# Specify the file path to the Excel file
dateipfad = r'C:\Users\chiar\OneDrive\Masterthesis\Data\LinkedIn\linkedin_companies_matched_experience.csv'

# Import Excel file
df_notcleaned = pd.read_csv(dateipfad)

# Access the imported data
df_notcleaned.head()


# 3. Data Preperation

In the course of data preperation, the data are first examined in general (3.1). Data outliers are checked (3.1.2), duplicates are checked (3.1.3) and the correlation is checked (3.1.4). It must be taken into account that the LinkedIn data has already been pre-cleaned before delivery. In the present notebook, it is nevertheless necessary to intensively clean data outliers (3.2.1), unneeded columns (3.2.2), zero values (3.2.4) and data formats (3.2.3, 3.2.5, 3.2.6, 3.2.7).

## 3.1 Data inspection

In [ ]:
df_notcleaned

Explanations to the columns:

- experience_index - an index of the experience within a member
- member_id - an employee unique identifier
- id - a unique identifier of the experience (of the row)
- location - location of the experience (As the user wrote it)
- company_name - name of the company on LinkedIn
- comapny_url - url of the company on LinkedIn
- date_form - date of starting the job (as the user specified. some have months, others only years). if "0" appears, that means this data is missing.
- date_to - date of finishing the job (as the user specified. some have months, others only years). if "0" appears, that means this data is missing.
- duration - calculated from date_to-date_from
- relevant - source of comparison, as specified in the file you sent me. note - the "not relevant" companies I kept (due to my explanation in the previous email) are of conf>0.95
- Firm_original_name - the original name of the firm in your data

**Finding:** Unnamed and id contain no added information and can therefore be droped. Location is probably also not relevant. 


In [ ]:
df_notcleaned.describe(include='all')

In [ ]:
unique_member_ids = df_notcleaned['member_id'].nunique()
print("Anzahl der eindeutigen Werte in der Spalte 'member_id':", unique_member_ids)

Unique values are for some rows high (company_name) because the names need to be matched to the firm original name. The original name is not available in all entries yet. Unique values of original is with 598 as expected.

**Finding:** company name, company url and firm original name display the same information. Therefore it needs to be tidyed up in one row and the remaining to needs to be droped.

In [ ]:
df_notcleaned.info()
#int = integers
#object = undefined format, can mean anything & is difficult to process
#float64 = decimal numbers, fraction

**Findings:** 
- date_from and date_to need to be converted to datetime64
- duration can remain float64

### 3.1.1 Checking for missing values

In [ ]:
df_notcleaned.isnull().sum()
#Note: is zero counts the values that are equal to missing or NaN. 

**Finding:** location can be droped since it containes many empty fields and the expected relevance to the use case is very low.

In [ ]:
for column in df_notcleaned.columns:
    count_filtered = (df_notcleaned[column].isin([0, '0.0'])).sum()
    print(f"{column}: {count_filtered}")

Experience index contains nulls. In context of contet comprehensible and therefore ok. Duration on the other hand should never be zero, since that indicates, that the position was never staffed.

**Findind:** Rows that contain null in duration needs to be removed. 

In [ ]:
# DataFrame is called df_notcleaned

# Output the entry in line 40321
row = df_notcleaned.loc[40321]
print(row)

test line shows that zeros are contained in date_to. It can be assumed that these entries are ongoing occupations. Therefore, the value "31-12-2018" is set in the data cleaning.  A similar check needs to be performed for the date_from column.

In [ ]:
# Count the number of entries with the text "0" in the column "date_to"/ "date_from"
count_zeros_text = (df_notcleaned['date_to'].astype(str) == "0").sum()
print(f"Number of entries with the text '0' in the column 'date_to': {count_zeros_text}")

count_zeros_text = (df_notcleaned['date_from'].astype(str) == "0").sum()
print(f"Number of entries with the text '0' in the column 'date_from': {count_zeros_text}")

As expected, the number of zeros is significantly lower for date_from. These lines should be dropped in cleaning process because there is no functional explanation for this.

### 3.1.2 Checking for data outliers

With the information about what each column says, it makes no sense to check each row for data outliers. Therefore, only the duration column is checked.

In [ ]:
selected_columns = ['duration']
plt.figure(figsize=(10, 8))
df_notcleaned[selected_columns].boxplot()
plt.title("Boxplot")
plt.show()

40000 days correspond to about 109 years. This is an unrealistic value. The average working life lasts about 40 years, i.e. about 15,000 working days. First, a 98 per cent quantile is formed. Then re-evaluate. 

### 3.1.3 Checking for dublicates

In [ ]:
duplicates = df_notcleaned[df_notcleaned.duplicated()]
print("Duplicate Rows : ",len(duplicates))
duplicates

As expected there are no dublicates in this dataset. No findings.

### 3.1.4 Check for correlation

In [ ]:
correlation_matrix = df_notcleaned.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Korrelationsmatrix")
plt.show()

Correlation of 0 does not represent a linear relationship. Basically, the results range around values close to zero. Only between unnamed and member_id is there a moderately strong positive correlation. Unnamed does not contain any additional value and is therefore removed in the course of the cleaning.

## What needs to be done to clean the data? A summary.

The following findings can be noted:

1. Clean data outliers in duration.
2. Remove columns unnamed and id. No added content. Also Remove location due to many missing fields and low business impact.
3. Replace 0 in date_to with 31-12-2018. Note that the duration is not recalculated (number of zeros will not reduce).
4. Drop all rows that contain 0 in Date_from.
5. Converte Date_from and Date_to to date values.
6. Calculate the duration in a new column. Check if column duration still contains zero.
7. Stock exchange abbreviations at the end of the company_original_name lead to problems with the naming of the dataframes. Therefore, a cleansed line must be created. The new line needs to be checked for special characters. 


The following points are left as they are. A clean-up would have the consequence that mappability would worsen and feature engeneering at employee level would become difficult.
- Use only one column for the identification of company (name/url/original name)
- Only keep companies that are mappable (Yes, additional)

In [ ]:
print('The dataset has {} rows and {} columns. This results in {} data entries.'.format(df_notcleaned.shape[0],df_notcleaned.shape[1], df_notcleaned.size)) 

Size is being used to generally identify if an cleaning action was successful.

## 3.2 Data Cleansing

A copy of the original dataset is created in a new dataframe to allow for backtracking and better control of the work status. rfp = ready for preperation

In [ ]:
df_rfp = df_notcleaned.copy()

In [ ]:
df_notcleaned.shape

### 3.2.1. Clean data outliers in duration

In [ ]:
quantile_98 = df_notcleaned['duration'].quantile(0.98)
df_notcleaned = df_notcleaned[df_notcleaned['duration'] <= quantile_98]

In [ ]:
df_notcleaned.shape

In [ ]:
selected_columns = ['duration']
plt.figure(figsize=(10, 8))
df_notcleaned[selected_columns].boxplot()
plt.title("Boxplot")
plt.show()

### 3.2.2 Remove columns unnamed, id and location

In [ ]:
df_rfp = df_rfp.drop(['Unnamed: 0', 'id', 'location'], axis=1)
df_rfp

### 3.2.3 Replace 0 in date_to with 31-12-2018

Reminder: Number of entries with the text '0' in the column 'date_to': 3140329 

In [ ]:
# Replace entries with the text "0" in the column "date_to" with the value "31-12-2018"
df_rfp.loc[df_rfp['date_to'].astype(str) == "0", 'date_to'] = "2018-12-31"

# Check the update
count_zeros_text_updated = (df_rfp['date_to'].astype(str) == "0").sum()
print(f"Number of entries with the text '0' in the column 'date_to' after the update: {count_zeros_text_updated}")


### 3.2.4 Delete rows with 0 in date_from

Reminder: Number of entries with the text '0' in the column 'date_from': 782209

In [ ]:
# Create a new DataFrame without the rows containing 0 in the "date_from" column
df_rfp2 = df_rfp[df_rfp['date_from'].astype(str) != "0"]

In [ ]:
count_zeros_text_updated = (df_rfp2['date_from'].astype(str) == "0").sum()
print(f"Number of entries with the text '0' in the column 'date_from' after the update: {count_zeros_text_updated}")

### 3.2.5 Converte columns date_from and date_to

In [ ]:
df_rfp3= df_rfp2.copy()

In [ ]:
value = df_rfp3.loc[37, 'date_to']
data_type = type(value)
print(data_type)

Indicates that values are shown as text.

In [ ]:
is_zero_text = df_rfp3['date_to'] == '0'
count_zero_text = is_zero_text.sum()
print("Number of '0' 'date_to':", count_zero_text)
is_zero_text = df_rfp3['date_from'] == '0'
count_zero_text = is_zero_text.sum()
print("Number of '0' 'date_from':", count_zero_text)

There are no missing values due to cleaning of duration.
Now it is necessary to clean up the formats that are in the YYYY format. Columns 36 and 37 contain such values in the original.

In [ ]:
df_rfp3['date_from'] = df_rfp3['date_from'].apply(lambda x: x + '-01-01' if len(str(x)) == 4 else x)

In [ ]:
print(df_rfp3['date_from'].head(40))

In [ ]:
df_rfp3['date_to'] = df_rfp3['date_to'].apply(lambda x: x + '-12-31' if len(str(x)) == 4 else x)

In [ ]:
print(df_rfp3['date_to'].head(40))

Cleaning successful. All data that had the format YYYY are now YYYY -01-01 in column date_from and YYYY 31-12 in date_to. Everything else remained the same.
Since the columns are now consistent, the entries can be converted from text format to datetime64 format. It is expected, that there are a few values that are not conform to YYYY-MM-DD. They are converted to NaN.

In [ ]:
df_rfp3['date_from'] = pd.to_datetime(df_rfp3['date_from'], format='%Y-%m-%d', errors='coerce')
df_rfp3['date_to'] = pd.to_datetime(df_rfp3['date_to'], format='%Y-%m-%d', errors='coerce')

In [ ]:
count_nat_date_from = df_rfp3['date_from'].isna().sum()
count_nat_date_to = df_rfp3['date_to'].isna().sum()

print(f"Number of NaT values in 'date_from': {count_nat_date_from}")
print(f"Number of NaT values in 'date_to': {count_nat_date_to}")

As expected very few entries. They are droped from the dataframe.

In [ ]:
print(df_rfp3.shape)

In [ ]:
df_rfp3.dropna(subset=['date_from', 'date_to'], inplace=True)
print(df_rfp3.shape)

In [ ]:
print(df_rfp3['date_from'].dtype)
print(df_rfp3['date_to'].dtype)

### 3.2.6 Exact calculation of date_from and calculation of duration where date_to was previously missing

In [ ]:
df_rfp4= df_rfp3.copy()

Since the date is now in the format datetime, the exact date_from can be calculated on the basis of the duration for those values where only a year number was present. 

In [ ]:
df_rfp4.insert(df_rfp4.columns.get_loc("date_from") + 1, "date_from_original", df_rfp4["date_from"].copy())
df_rfp4.head(3)

In [ ]:
mask = (df_rfp4['duration'] != 0.0) & (df_rfp4['duration'] != 0)
df_rfp4.loc[mask, 'date_from'] = df_rfp4.loc[mask, 'date_to'] - pd.to_timedelta(df_rfp4.loc[mask, 'duration'], unit='D')
df_rfp4.head(40)

For those lines where the date_to value was missing, the duration is now calculated.

In [ ]:
df_rfp4.insert(df_rfp4.columns.get_loc("duration") + 1, "duration_original", df_rfp4["duration"].copy())
df_rfp4.head(3)

In [ ]:
df_rfp4['duration'] = (df_rfp4['date_to'] - df_rfp4['date_from']).dt.days
df_rfp4.head(40)

With lines 35 and 26, it can be determined that the cleaning was successful. The two copied columns can be removed again.

In [ ]:
df_rfp4.drop(['date_from_original', 'duration_original'], axis=1, inplace=True)
df_rfp4.head(2)

In [ ]:
df_rfp4.shape

In [ ]:
count_filtered = (df_rfp4['duration'].astype(int) == 0).sum()
print(f"Count: {count_filtered}")

There are still 405.902 values with null. Since those values do not add value they are removed from the dataframe. Rows should be reduced by this size.

In [ ]:
value = df_rfp4.loc[1, 'duration']
data_type = type(value)
print(data_type)

In [ ]:
df_rfp4 = df_rfp4[df_rfp4['duration'] != 0]
print(df_rfp4['duration'].head(2))

In [ ]:
num_rows, num_columns = df_rfp4.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

### 3.2.7 Cleaning Firm_original_name

In [ ]:
 df_rfp5= df_rfp4.copy()

In [ ]:
# Copy the column
df_rfp5['Copy_Firm_original_name'] = df_rfp5['Firm_original_name']

# Remove values in brackets
df_rfp5['Firm_original_name'] = df_rfp5['Firm_original_name'].apply(lambda x: re.sub(r'\(.*\)', '', str(x)).strip())

df_rfp5.head(40)

Checking for special characters. Special characters are removed from the respective name.

In [ ]:
df_filtered = df_rfp5[df_rfp5['Firm_original_name'].str.contains(r'[^\w\s]', regex=True)]
df_filtered.shape

In [ ]:
# Function for cleaning  Firm_original_name
def clean_firm_name(name):
    return re.sub(r'[^\w\s]', '', str(name))

# clean Firm_original_name 
df_rfp5['Firm_original_name'] = df_rfp5['Firm_original_name'].apply(clean_firm_name)


In [ ]:
df_filtered = df_rfp5[df_rfp5['Firm_original_name'].str.contains(r'[^\w\s]', regex=True)]
df_filtered.shape

In [ ]:
num_duplicates_a = df_rfp5.duplicated().sum()
num_duplicates_a

In [ ]:
df_rfpfinal= df_rfp5.copy()

## 3.3 Feature engeneering & strucuturing the data as needed

In the following, aggegated features are formed from the individual profile information. In order to extract new features from the raw data, the data must first be prepared beyond the classic data preperation.

In [ ]:
#Creating a copy for better work controll --> df_rfe = ready for engeneering
df_rfe1= df_rfpfinal.copy()

### 3.3.1 Prepering for engeneering

#### Step 1: filtering the relevant timeline (2014 - 2018)

The dataset df_rfe1 is still very extensive and contains data on employees who worked for one of the relevant companies before 2014. These employees are irrelevant for further analysis. Therefore, entries for the period between 2014 and 2018 are filtered first and then a dataframe is formed that contains all entries on employees who worked for a relevant company in the period mentioned. Note: relevant is not yet cleansed here.

In [ ]:
# Step 1: Identify entries with period between 2014 and 2018
date_from = pd.to_datetime(df_rfe1['date_from'])
date_to = pd.to_datetime(df_rfe1['date_to'])
mask = (date_from.dt.year >= 2014) & (date_from.dt.year <= 2018) & (date_to.dt.year >= 2014) & (date_to.dt.year <= 2018)
relevant_entries = df_rfe1[mask]

# Step 2: Check condition Firm_original_name > 2 characters
relevant_entries = relevant_entries[relevant_entries['Firm_original_name'].str.len() > 2]

# Step 3: Remember the Member IDs
relevant_member_ids = relevant_entries['member_id'].unique()

# Step 4: Create new dataframe df_rfe2 with the identified member IDs
df_rfe2 = df_rfe1[df_rfe1['member_id'].isin(relevant_member_ids)].copy()


In [ ]:
df_rfe2

In [ ]:
if 266 in df_rfe2['member_id'].values:
    print("266 contained in df")
else:
    print("266 not contained in df")


Removal of entries outside 2014 -2018 successful. Member ID 266 serves as a sample. Relevant work experience here was 1988. 

#### Step 2: Mapping between company and member id

At this point, all companies that are not clearly mappable are removed from the data set. Only those companies that have Yes or additional in the relevant column are retained. Subsequently, the employees (member_id) are assigned to the respective companies. This step is necessary because in order to reduce the size and optimise performance, the currently used data set df_rfe2 was reduced by the entries that have no entry in the column Firm_original_name. However, the information about the individual employees will be relevant again later.

In [ ]:
#Forming new dataframe that onnly containes Yes and additional values
df_yes_additional = df_rfe2[df_rfe2['relevant'].isin(['Yes', 'additional'])]
df_yes_additional

In [ ]:
# Filter the lines where the Firm_original_name is "Prudential plc".
filtered_rows = df_yes_additional[df_yes_additional["Firm_original_name"] == "Prudential plc"]

# Print
print(filtered_rows)

In [ ]:
# Create an empty dictionary to store the assignment of Firm_original_name to Member-IDs
member_id_dict = {}

# Iterate over each row of the DataFrame df_yes_additional
for index, row in df_yes_additional.iterrows():
    # Extract the Firm_original_name and the Member-ID of the current line
    firm_name = row['Firm_original_name']
    member_id = row['member_id']
    
    # Add the member ID to the corresponding Firm_original_name in the dictionary
    member_id_dict.setdefault(firm_name, []).append(member_id)

# Output the assignment of Firm_original_name to Member IDs
for firm_name, member_ids in list(member_id_dict.items())[:2]:
    print(f"Firm_original_name: {firm_name}, Member-IDs: {member_ids}")


In [ ]:
# Iterate over each item in the member_id_dict
for firm_name, member_ids in member_id_dict.items():
    # Remove duplicate member_ids
    unique_member_ids = list(set(member_ids))
    
    # Update the member_ids in the member_id_dict
    member_id_dict[firm_name] = unique_member_ids


In [ ]:
# Get the first company from member_id_dict
first_company = next(iter(member_id_dict.keys()))

# Get the member IDs for the first company
member_ids = member_id_dict[first_company]

# Print the first company and its member IDs
print(f"First Company: {first_company}")
print(f"Member IDs: {member_ids}")

#### Step 3: Extract member_id data for each company 
A separate data frame is created for each company in the column Firm_original_name. This contains all entries on the employees who worked for the company in the period between 2014 and 2018.

In [ ]:
# Output all unique values in the column "Firm_name_original" in the desired format
unique_firm_names = df_rfe2['Firm_original_name'].unique()
formatted_firm_names = ', '.join(['"' + name + '"' for name in unique_firm_names])
print(formatted_firm_names)

In [ ]:
# Extract the first 200,000 entries of the DataFrame df_rfe2
df_subset = df_rfe2.head(200000)

# Create the directory path and file name
csv_filename = r'C:\Users\wildn\Downloads\Master\Debug.csv'

# Save the Subset DataFrame as CSV
df_subset.to_csv(csv_filename, index=False)


In [ ]:
# Iterate over each unique firm_name in df_rfe2 and create separate CSV files
for firm_name in df_yes_additional['Firm_original_name'].unique():
    # Get the associated member_id list from member_id_dict for the current firm_name
    member_ids = member_id_dict.get(firm_name, [])
    
    # Filter the rows for the current firm_name using the member_id list
    company_dataframe = df_rfe2[df_rfe2['member_id'].isin(member_ids)].copy()

    # Print the name of the created DataFrame
    print(f"Created DataFrame for: {firm_name}")

    # Create the directory path and filename
    csv_filename = os.path.join(r'C:\Users\wildn\Downloads\Master\company_dictonary', f"df_{firm_name}.csv")

    # Save the DataFrame as CSV
    company_dataframe.to_csv(csv_filename, index=False)

In [ ]:
# Create an empty dictionary to store the dataframes
company_dataframes = {}

# Directory path containing the CSV files
directory_path = r'C:\Users\wildn\Downloads\Master\company_dictonary'

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    # Check if the file is a CSV file
    if filename.endswith('.csv'):
        # Generate the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Read the CSV file and create a DataFrame
        df = pd.read_csv(file_path)
        
        # Extract the DataFrame name from the file name (without extension .csv)
        dataframe_name = os.path.splitext(filename)[0]
        
        # Add the DataFrame to the company_dataframes dictionary with the filename as the key
        company_dataframes[dataframe_name] = df

        # Print the name of the loaded DataFrame
        print(f"Loaded DataFrame: {dataframe_name}")

In [ ]:
# Remove 'df_nan' from the dictionary if it exists
company_dataframes.pop('df_nan', None)

The company Ferrovial S.A. will be used to validate code results to ensure accuracy.

In [ ]:
company_dataframes['df_Ferrovial  SA']

In [ ]:
num_duplicates = company_dataframes['df_Ferrovial  SA'].duplicated().sum()
num_duplicates

### 3.3.2 Feature engeneering

#### Feature 1: Calculation of the employees' work experience at annual level
Work experience is calculated on the basis of the duration column. For this purpose, the sum of the entries for duration from the previous professional stations is formed and divided by 365. The column Years of experience shows how much work experience the employee had when he was hired.

In [ ]:
# Create an empty column "Years of Experience" in each DataFrame of the company_dataframes-Dictionary
for df_name, df in company_dataframes.items():
    df['Years of Experience'] = ''

    # Sort the current DataFrame by member_id and date_from
    df = df.sort_values(['member_id', 'date_from'])

    # Iterate over each unique member_id in the current DataFrame
    for member_id in df['member_id'].unique():
        # Filtere den aktuellen DataFrame nach der aktuellen member_id
        member_id_df = df[df['member_id'] == member_id]

        # Calculate work experience for each row
        previous_experience = 0
        previous_duration = 0
        for index, row in member_id_df.iterrows():
            duration = row['duration']

            # Check if it is the first entry
            if previous_duration != 0:
                years_of_experience = (previous_experience + previous_duration) / 365
                df.at[index, 'Years of Experience'] = years_of_experience

            previous_experience += previous_duration
            previous_duration = duration

    # Update the DataFrame in the company_dataframes dictionary
    company_dataframes[df_name] = df

    # Show the modified DataFrame with the new "Years of Experience" column
    print(f"DataFrame: {df_name}")

In [ ]:
df_filtered = company_dataframes['df_Ferrovial  SA'][company_dataframes['df_Ferrovial  SA']['member_id'] == 3963].sort_values('date_from')
df_filtered

#### Feature 2: Employee development

In order to be able to track the development of employees over the years 2014 - 2018, it is necessary to break down to the annual level how many employees were employed and when. For the Migration of Work Experience feature, it will be relevant to know which employee worked when. Therefore, the member_id is used to determine which periods of activity exist. This is done for each dataframe of the respective companies.

In [ ]:
# Extract Year
# Iterate over each DataFrame in the company_dataframes dictionary.
for df_name, df in company_dataframes.items():
    # Convert "date_from" column to datetime64 format
    df['date_from'] = pd.to_datetime(df['date_from'], format='%Y-%m-%d')
    
    # Convert "date_to" column to datetime64 format
    df['date_to'] = pd.to_datetime(df['date_to'], format='%Y-%m-%d')

    # Extract the year from the "date_from" column
    df['Year date_from'] = df['date_from'].dt.year
    
    # Extract the year from the "date_to" column
    df['Year date_to'] = df['date_to'].dt.year
    
    # Update DataFrame in company_dataframes dictionary
    company_dataframes[df_name] = df
    
    # Output name of edited DataFrame
    print(f"DataFrame: {df_name}")

In [ ]:
print(company_dataframes['df_Ferrovial  SA']['date_from'].dtype) 

In [ ]:
company_dataframes['df_Ferrovial  SA']

In [ ]:
# Introducing column Years total to have the Years at one place
# Iterate over each DataFrame in the company_dataframes dictionary
for df_name, df in company_dataframes.items():
    # Iterate over each row in the current DataFrame
    for index, row in df.iterrows():
        years_range = list(range(row['Year date_from'], row['Year date_to'] + 1))
        years_total = list(set(years_range + [row['Year date_from'], row['Year date_to']]))
        df.at[index, 'Years total'] = ','.join(map(str, years_total))

    # Update the DataFrame in the company_dataframes dictionary
    company_dataframes[df_name] = df
    
    # Print edited dfs
    print(f"DataFrame: {df_name}")

In [ ]:
company_dataframes['df_Ferrovial  SA']

In [ ]:
# Iterate over each DataFrame in the company_dataframes dictionary.
for df_name, df in company_dataframes.items():
    # 1. filter by the name of the company
    firm_name = df_name.replace("df_", "")  # Entferne "df_" aus dem DataFrame-Namen
    df['Firm_original_name'] = df['Firm_original_name'].fillna('')  # Replace NaN with an empty string
    df.loc[df['Firm_original_name'].str.contains(firm_name), 'Firm_original_name'] = firm_name

    # 2. Add columns for years 2014-2018
    for year in range(2014, 2019):
        df[str(year)] = 0

    # 3. Set 1 in the corresponding years in which the Member ID has worked
    for index, row in df.iterrows():
        if row['Firm_original_name'] == firm_name:
            years_worked = list(map(int, row['Years total'].split(',')))
            for year in range(2014, 2019):
                if year in years_worked:
                    df.at[index, str(year)] = 1
                else:
                    df.at[index, str(year)] = 0

    # Update the DataFrame in the company_dataframes dictionary
    company_dataframes[df_name] = df

    # Print edited dfs
    print(f"DataFrame: {df_name}")


Validation of the code with Ferrovial:

In [ ]:
company_dataframes['df_Ferrovial  SA'].head(10)

In [ ]:
filtered_df = company_dataframes['df_Ferrovial  SA'][company_dataframes['df_Ferrovial  SA']['Firm_original_name'] == 'Ferrovial  SA']
filtered_df 

In [ ]:
file_path = r'C:\Users\wildn\Downloads\Master\KLA Corporation.csv'
company_dataframes['df_KLA Corporation'].to_csv(file_path, index=False)

Spot checks in df for Ferrovial successful. The allocation in time slices was done correctly.

A new dataframe is created. This will be used to display the aggregated features of the individual companies. The dataframe is called df_waf (= _with aggregated features).

In [ ]:
# Extract unique values from df_rfe1 column Firm_original_name
unique_firms = df_rfe1['Firm_original_name'].unique()

# Create DataFrame df_waf
df_waf = pd.DataFrame(columns=['Firm_original_name'])

# Fill Firm_original_name column with unique values
df_waf['Firm_original_name'] = unique_firms

# Add columns for the years 2014-2018 and initialise with 0
for year in range(2014, 2019):
    df_waf[str(year)] = 0

df_waf

All member_ids that show a 1 in the respective column in the company dataframe are written into the columns for the years 2014 to 2018. Duplicate ids are removed.

In [ ]:
# 1. Edit all DataFrames in the company_dataframes dictionary
for df_name, df in company_dataframes.items():
    # 2. Go through years 2014-2018
    for year in range(2014, 2019):
        # 3. Filter to 1 in the corresponding column
        filtered_df = df[df[str(year)] == 1]
        
        # 4. Identify the values in the column 'member_id'.
        member_ids_year = filtered_df['member_id'].tolist()
        
        # 5. Transfer the member_ids into the DataFrame df_waf into the corresponding column
        firm_name = df_name.replace("df_", "")  # Remove 'df_' from the DataFrame name
        row_index = df_waf[df_waf['Firm_original_name'] == firm_name].index[0]
        existing_member_ids = df_waf.at[row_index, str(year)]
        new_member_ids = set(member_ids_year)  # Remove duplicate member_ids
        if existing_member_ids:
            existing_member_ids = existing_member_ids.split(',')  # Break down the existing member IDs into a list
            new_member_ids.update(existing_member_ids)  # Add existing member_ids
        df_waf[str(year)] = df_waf[str(year)].astype('object')  # Make sure that the corresponding column has the data type 'object'.
        df_waf.at[row_index, str(year)] = ','.join(str(id_) for id_ in new_member_ids)  # Update df_waf

df_waf

In [ ]:
# Identify the rows with empty values in the "2014" column
mask_null_or_empty = df_waf["2014"].isnull() | df_waf["2014"].eq("")

# Filter the rows that contain empty values in the "2014" column and output the "Firm_original_name".
rows_with_null_or_empty = df_waf[mask_null_or_empty]
firm_names_with_null_or_empty = rows_with_null_or_empty["Firm_original_name"]

print(f"The following company names have empty values in the '2014' column:")
print(firm_names_with_null_or_empty)

In [ ]:
ferrovial_index = df_waf[df_waf['Firm_original_name'] == 'Ferrovial  SA'].index[0]
values_2015 = df_waf.at[ferrovial_index, '2015']
print(values_2015)

By counting the individual member ids, the number of member ids can be determined. 

In [ ]:
# Go through years 2014-2018
for year in range(2014, 2019):
    # Count the number of member IDs in the corresponding column for each row
    column_name = str(year)
    new_column_name = f'Number of employees {year}'
    df_waf[new_column_name] = df_waf[column_name].apply(lambda x: len(str(x).split(',')))

df_waf

Validation of the results: The original data frame is accessed and evaluated to see how many member ids are present and how many dubilakte are contained. The difference should correspond to the number in df_waf.

In [ ]:
columns = ['2014', '2015', '2016', '2017', '2018']

# Filter DataFrame company_dataframes['df_Ferrovial  S.A.'] for columns
filtered_df = company_dataframes['df_Ferrovial  SA'][columns]

# Count 1 in selected columns
occurrences = filtered_df.eq(1).sum()

print(occurrences)

In [ ]:
# 1. Filter for company name and column 2014
filtered_df = company_dataframes['df_Ferrovial  SA'][(company_dataframes['df_Ferrovial  SA']['Firm_original_name'] == 'Ferrovial  SA') & (company_dataframes['df_Ferrovial  SA']['2014'] == 1)]

# 2. Filter dublicate member-IDs in column 'member_id' and count frequency
duplicate_member_ids = filtered_df['member_id'].value_counts()
duplicate_member_ids = duplicate_member_ids[duplicate_member_ids > 1]

# 3. Calculate sum of frequency
total_duplicates_sum = duplicate_member_ids.sum()

# Results
print("Frequency of dublicate member-IDs  2014:")
print(duplicate_member_ids)
print("Frequency total:")
print(total_duplicates_sum)

Check successful: There are 815 entries for the year 2014, of which 181 are duplicates. In order not to eliminate duplicates completely, length must be added again with 87. 

815-181+87= 721

Now Feature Employee development can be created for each year:

In [ ]:
# Calculate and populate "Employee development" column for the year 2015
df_waf['Employee development 2015'] = ((df_waf['Number of employees 2015'] - df_waf['Number of employees 2014']) / df_waf['Number of employees 2014']) * 100

# Calculate and populate "Employee development" column for the year 2016
df_waf['Employee development 2016'] = ((df_waf['Number of employees 2016'] - df_waf['Number of employees 2015']) / df_waf['Number of employees 2015']) * 100

# Calculate and populate "Employee development" column for the year 2017
df_waf['Employee development 2017'] = ((df_waf['Number of employees 2017'] - df_waf['Number of employees 2016']) / df_waf['Number of employees 2016']) * 100

# Calculate and populate "Employee development" column for the year 2018
df_waf['Employee development 2018'] = ((df_waf['Number of employees 2018'] - df_waf['Number of employees 2017']) / df_waf['Number of employees 2017']) * 100

In [ ]:
df_waf

#### Feature 3: Calculation of notices in each year

To calculate notices the member Ids from previous year were searched in current year. Those who could not be found were identfied as employees who left the organisation.

In [ ]:
# Iterate over each year (2014-2018)
for year in range(2014, 2019):
    current_year_col = str(year)
    next_year_col = str(year + 1)
    notices_col = f'Notices {year}'
    
    # Create a new column for notices
    df_waf[notices_col] = ''
    
    # Check if the next year column exists
    if next_year_col in df_waf.columns:
        # Iterate over each row in the dataframe
        for index, row in df_waf.iterrows():
            current_year_ids = set(str(row[current_year_col]).split(','))
            next_year_ids = set(str(row[next_year_col]).split(','))

            missing_ids = []

            # Iterate over each ID in the current year's IDs
            for member_id in current_year_ids:
                if member_id not in next_year_ids:
                    missing_ids.append(member_id)

            # Join the missing IDs into a comma-separated string
            missing_ids_str = ','.join(missing_ids)

            # Update the notices column for the current year
            df_waf.at[index, notices_col] = missing_ids_str 
    else:
        # If the next year column doesn't exist, set the notices column as 'Not available'
        df_waf[notices_col] = '0'


In [ ]:
df_waf

In [ ]:
# Iterate over the years 2014-2018
for year in range(2014, 2019):
    # Count the number of member_ids in the corresponding column for each row
    column_name = f'Notices {year}'
    new_column_name = f'Number of notices {year}'
    df_waf[new_column_name] = df_waf[column_name].apply(lambda x: len(str(x).split(',')))

df_waf["Number of Notices 2018"] = 0

# Print
df_waf

In [ ]:
# Get the value in the "2014" column of the 3rd row
value_2014 = df_waf.loc[3, "Notices 2014"]

# Print the value
print(f"Value in '2014' column of the 3rd row: {value_2014}")


#### Feature 4: Calculation of migration work expierince due to notices

In der Spalte Notices Year habe ich die Information welche member_id gegangen ist. In dem df_company habe ich das year_date to und die Spalte Years of experience 

In [ ]:
# Go through 2014-2018
for year in range(2014, 2019):
    year_col = str(year)
    migrating_col = f'Migrating work experience {year}'

    # Add new column "Migrating work experience <Year>" 
    df_waf[migrating_col] = 0.0

df_waf

In [ ]:
# Dictionary to store the final results per year
results = {}

# Iterate over the years 2014 to 2018
for firm_name in df_waf["Firm_original_name"].unique():
    # Go through years 2014 to 2018
    for year in range(2014, 2019):
       # 2. Extract the member_ids from the corresponding column Notices <Year>
        notices_column = f"Notices {year}"
        member_ids = df_waf.loc[df_waf["Firm_original_name"] == firm_name, notices_column].str.split(",").explode()

        # Remove empty or invalid values
        member_ids = member_ids[member_ids != ''].astype(int)

        # 3. Check if there is a corresponding DataFrame in the company_dataframes dictionary
        dataframe_key = f"df_{firm_name}"
        if dataframe_key not in company_dataframes:
            print(f"Dataframe '{dataframe_key}' nicht gefunden. Überspringe Prüfung für '{firm_name}' im Jahr {year}.")
            continue

        df_firm = company_dataframes[dataframe_key]

        # 4. Perform actions for the found DataFrame
        years_of_experience_sum = 0

        for member_id in member_ids:
            # 4.1 Filter by member_id and sort by date_from
            filtered_df = df_firm[df_firm["member_id"] == member_id].sort_values("date_from")

            for index, row in filtered_df.iterrows():
                # 4.3 Check for the year in the Year date_to column
                if row["Year date_to"] == year:
                    # 4.4 Check for a higher date in the Year date_to column
                    next_index = index + 1
                    if next_index < len(filtered_df) and filtered_df.loc[next_index, "Year date_to"] > year:
                        # 4.4.2 Calculate the duration in years and add to years_of_experience_sum
                        duration_years = filtered_df.loc[index, "duration"] / 365
                        years_of_experience = row["Years of Experience"]
                        if years_of_experience != '':
                            years_of_experience_sum += duration_years + float(years_of_experience)
                    else:
                        # 4.4.1 Store value from the Years of Experience column
                        years_of_experience = row["Years of Experience"]
                        if years_of_experience != '':
                            years_of_experience_sum += float(years_of_experience)

        # 4.6 Calculate final result
        number_of_notices = df_waf.loc[df_waf["Firm_original_name"] == firm_name, f"Number of notices {year}"].values[0]
        migrating_work_experience = years_of_experience_sum / number_of_notices

        # Save result in results-Dictionary
        results[(firm_name, year)] = migrating_work_experience

# 5. Check and update the entries in the df_waf DataFrame
for (firm_name, year), value in results.items():
    column_name = f"Migrating work experience {year}"
    df_waf.loc[df_waf["Firm_original_name"] == firm_name, column_name] = value

In [ ]:
df_waf

Random validation confirms the order of magnitude of the figures.

#### Feature 5: Calcuating Number of new joiners and work experience they bring along 

In [ ]:
# Iterate over each year (2014-2018)
for year in range(2014, 2019):
    current_year_col = str(year)
    previous_year_col = str(year - 1)
    new_joiners_col = f'New joiners {year}'
    
    # Create a new column for new joiners
    df_waf[new_joiners_col] = ''
    
    # Check if the previous year column exists
    if previous_year_col in df_waf.columns:
        # Iterate over each row in the dataframe
        for index, row in df_waf.iterrows():
            current_year_ids = set(str(row[current_year_col]).split(','))
            previous_year_ids = set(str(row[previous_year_col]).split(','))

            missing_ids = []

            # Iterate over each ID in the current year's IDs
            for member_id in current_year_ids:
                if member_id not in previous_year_ids:
                    missing_ids.append(member_id)

            # Join the missing IDs into a comma-separated string
            missing_ids_str = ','.join(missing_ids)

            # Update the new joiners column for the current year
            df_waf.at[index, new_joiners_col] = missing_ids_str 
    else:
        # If the previous year column doesn't exist, set the new joiners column as '0'
        df_waf[new_joiners_col] = '0'

In [ ]:
df_waf

In [ ]:
# Iterate over the years 2014-2018
for year in range(2014, 2019):
    # Count the number of member IDs in the corresponding column for each row
    column_name = f'New joiners {year}'
    new_column_name = f'Number of New Joiners {year}'
    df_waf[new_column_name] = df_waf[column_name].apply(lambda x: len(str(x).split(',')))

df_waf["Number of New Joiners 2014"] = 0

# Print
df_waf

In [ ]:
# Dictionary to store the final results per year
results = {}

# Iterate over each unique company name in the "Firm_original_name" column of the df_waf DataFrame
for firm_name in df_waf["Firm_original_name"].unique():
    # Iterate over each year from 2014 to 2018
    for year in range(2014, 2019):
        # Extract the member IDs from the corresponding "New joiners <Year>" column and store them in the member_ids variable
        new_joiners_column = f"New joiners {year}"
        member_ids = df_waf.loc[df_waf["Firm_original_name"] == firm_name, new_joiners_column].str.split(",").explode()

        # Remove empty or invalid values from the member IDs
        member_ids = member_ids[member_ids != ''].astype(int)

        # Check if there is a corresponding DataFrame for the company in the company_dataframes dictionary
        dataframe_key = f"df_{firm_name}"
        if dataframe_key not in company_dataframes:
            print(f"Dataframe '{dataframe_key}' not found. Skipping check for '{firm_name}' in year {year}.")
            continue

        # Retrieve the DataFrame for the company and store it in the df_firm variable
        df_firm = company_dataframes[dataframe_key]

        # Initialize the sum for years of experience (years_of_experience_sum)
        years_of_experience_sum = 0

        # Iterate over each member ID
        for member_id in member_ids:
            # Filter by member ID and sort by the start date ("date_from")
            filtered_df = df_firm[df_firm["member_id"] == member_id].sort_values("date_from")

            # Iterate over each row in the filtered DataFrame
            for index, row in filtered_df.iterrows():
                # Check if the year in the "Year date_from" column matches the current year
                if row["Year date_from"] == year:
                    # Take the value from the "Years of Experience" column and add it to years_of_experience_sum
                    years_of_experience = row["Years of Experience"]
                    if years_of_experience != '':
                        years_of_experience_sum += float(years_of_experience)

        # Calculate the final result, the average work experience per New Joiner
        number_of_new_joiners = df_waf.loc[df_waf["Firm_original_name"] == firm_name, f"Number of New Joiners {year}"].values[0]
        joining_work_experience = years_of_experience_sum / number_of_new_joiners

        # Save the result in the results dictionary using the company name and year as the key
        results[(firm_name, year)] = joining_work_experience

# Check and update the entries in the df_waf DataFrame
for (firm_name, year), value in results.items():
    column_name = f"New joining work experience {year}"
    df_waf.loc[df_waf["Firm_original_name"] == firm_name, column_name] = value

In [ ]:
df_waf

#### Feature 6: Fluctuation rate

The employee turnover rate is defined as follows: Number of employee departures divided by the average number of employees multiplied by 100.

In [ ]:
years = range(2014, 2019)

# Iterate over each year
for year in years:
    # Create the column name for the fluctuation rate
    column_name = f"Fluctuation rate {year}"
    
    # Calculate the fluctuation rate for the current year
    # by dividing the number of notices by the number of employees and multiplying by 100
    df_waf[column_name] = (df_waf[f"Number of notices {year}"] / df_waf[f"Number of employees {year}"]) * 100
df_waf

As the number of employees registered on LinkedIn does not correspond to the total number of employees, this figure should be viewed with caution. Calculation correct.

#### Feature 7: Average average length of service with the company

The average work experience can be calculated by restricting the company dataframe in the column Firm_original_name to the searched company. Then the sum of the entries in the column duration is calculated and divided by 365 to obtain years. This number is divided by the number of unique values in the column member_id.

In [ ]:
# Create the column "Average years of service with the company" in the df_waf DataFrame
df_waf["Average years of service with the company"] = ""

# Iterate over each unique company name in the "Firm_original_name" column of the df_waf DataFrame
for firm_name in df_waf["Firm_original_name"].unique():
    # Check if there is a corresponding DataFrame for the company in the company_dataframes dictionary
    dataframe_key = f"df_{firm_name}"
    if dataframe_key in company_dataframes:
        # Filter the company dataframe by the current company name in the "Firm_original_name" column
        company_df = company_dataframes[dataframe_key].loc[company_dataframes[dataframe_key]["Firm_original_name"] == firm_name]
        
        # Sum the values in the "duration" column of the filtered list
        duration_sum = company_df["duration"].sum()
        
        # Divide the sum by 365 to get an intermediate result
        intermediate_result = duration_sum / 365
        
        # Get the number of unique values in the "member_id" column of the filtered list
        unique_member_ids = company_df["member_id"].nunique()
        
        # Check if unique_member_ids is greater than 0 to avoid division by zero
        if unique_member_ids > 0:
            # Calculate the final result by dividing the intermediate result by the number of unique member IDs
            final_result = intermediate_result / unique_member_ids
        else:
            # Set a default value for the final result if unique_member_ids is 0
            final_result = 0.0  # Change this to any appropriate default value
        
        # Write the final result into the "Average years of service with the company" column for the current company row
        df_waf.loc[df_waf["Firm_original_name"] == firm_name, "Average years of service with the company"] = final_result

In [ ]:
df_waf

Company affiliations were validated using Ferrioval S.A. as an example. Based on the dataframe, 3.2 years of average work experience were also calculated in Excel.

#### Feature 8: Employees who worked for the company more than once or in diffrent positions

In [ ]:
# Step 1: Iterate over each company name
for firm_name in df_waf['Firm_original_name']:
    # Step 2: Check whether a corresponding DataFrame is available
    if f'df_{firm_name}' in company_dataframes:
        # Step 3: Filter the DataFrame by company name
        filtered_df = company_dataframes[f'df_{firm_name}'][company_dataframes[f'df_{firm_name}']['Firm_original_name'] == firm_name]
        
        # Step 4: Count the number of member_ids that occur more than once
        count = filtered_df['member_id'].duplicated().sum()
        
        # Step 5: Insert the result of the counter into df_waf
        df_waf.loc[df_waf['Firm_original_name'] == firm_name, 'More than once/different position'] = count

df_waf

1060 can be validated as correct.

In [ ]:
# Create the directory
directory_path = r'C:\Users\wildn\Downloads\Master'

# Name
filename = 'df_waf.tsv'

# Create the full file path
tsv_filepath = os.path.join(directory_path, filename)

# Save the DataFrame as a TSV file
df_waf.to_csv(tsv_filepath, sep='\t', index=False)

In [ ]:
df_waf_final = df_waf.copy()

In [ ]:
columns_to_drop = ['2014', '2015', '2016', '2017', '2018',
                   'Notices 2014', 'Notices 2015', 'Notices 2016', 'Notices 2017', 'Notices 2018',
                   'New joiners 2014', 'New joiners 2015', 'New joiners 2016', 'New joiners 2017', 'New joiners 2018']

df_waf_final.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# Create the directory
directory_path = r'C:\Users\wildn\Downloads\Master'

# Namw
filename = 'df_waf_final.csv'

# Create the full file path
csv_filepath = os.path.join(directory_path, filename)

# Save the DataFrame as CSV with the chosen separator and adjusted decimal separator
df_waf_final.to_csv(csv_filepath, sep=';', index=False, decimal=',')


In [ ]:
print(df_waf_final.dtypes)

In [ ]:
# Access to a specific cell in the column "column_name" and row 0
value = df_waf.loc[1, "Employee development 2016"]
print(value)
